# Control of a "spacecraft" platform with a reaction wheel

In [ ]:
# These are standard modules
import time
import numpy as np
from scipy import linalg
import sympy as sym
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown

# This is a custom interface to the pybullet simulator
import ae353_platform

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

Given a choice of ground roll angle $\phi$, the equations of motion are

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1)\sin(\phi) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)\sin(\phi)
\end{aligned}
$$

where

* $q_1$ is the platform angle
* $\dot{q}_1$ is the platform velocity
* $v_2$ is the wheel velocity
* $\tau$ is the torque applied *to* the platform *by* the wheel

and

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

and parameter values are defined as follows:

In [ ]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

# gravity
g = 9.81

# composite parameters
J1 = Jp + mw * l**2
J2 = Jw

The equations of motion when $\phi=\pi/2$ are

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1).
\end{aligned}
$$

We can write these equations of motion in standard form as

$$
\begin{bmatrix} \dot{q}_1 \\ \dot{v}_1 \\ \dot{v}_2 \end{bmatrix}
=
f \left( \begin{bmatrix} q_1 \\ v_1 \\ v_2 \end{bmatrix}, \begin{bmatrix} \tau \end{bmatrix} \right).
$$

Create a simulator for this case.

In [ ]:
simulator = ae353_platform.Simulator(
    display=True,
    roll=(np.pi / 2),
)

Choose a default camera view (could also be `simulator.camera_topview()`).

In [ ]:
simulator.camera_sideview()

Define a function to return a state-space model for a given choice of equilibrium point.

In [ ]:
def get_model(q1e, v1e, v2e, taue):
    # Define variables
    q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau')
    
    # Define right-hand side of ODEs
    f = sym.Matrix([
        v1,
        (1 / J1) * (tau - mw * g * l * sym.sin(q1)),
        (1 / J2) * (- ((J1 + J2) / J1) * tau + (J2 / J1) * mw * g * l * sym.sin(q1))
    ])

    # Convert floating-point to rational numbers
    f = sym.nsimplify(f, rational=True)
    
    # Verify equilibrium point
    f_num = sym.lambdify([q1, v1, v2, tau], f)
    if not np.allclose(f_num(q1e, v1e, v2e, taue), 0.):
        raise Exception('equilibrium point is invalid')
    
    # Find A and B in symbolic form
    A_sym = f.jacobian([q1, v1, v2])
    B_sym = f.jacobian([tau])

    # Create lambda functions to allow numerical evaluation of A and B
    A_num = sym.lambdify([q1, v1, v2, tau], A_sym)
    B_num = sym.lambdify([q1, v1, v2, tau], B_sym)

    # Find A and B in numeric form (making sure the result is floating-point)
    A = A_num(q1e, v1e, v2e, taue).astype(float)
    B = B_num(q1e, v1e, v2e, taue).astype(float)

    # Return A and B
    return A, B

Choose equilibrium point.

In [ ]:
q1e = np.pi
v1e = 0.
v2e = 0.
taue = 0.

Get state-space model.

In [ ]:
A, B = get_model(q1e, v1e, v2e, taue)

print(f'A =\n{A}\n\nB =\n{B}')

Check if the system is controllable.

In [ ]:
W = np.block([B, A @ B, A @ A @ B])
print(f'W =\n{W}\n')
print(f'rank(W) = {np.linalg.matrix_rank(W)}')

This function returns the solution to the LQR problem

$$\begin{align*} \underset{u_{[t_0,\infty]}}{\text{minimize}} &\qquad\int_{t_0}^{\infty} \left( x(t)^T Q x(t) + u(t)^T R u(t) \right) dt\\ \text{subject to} &\qquad\dot{x}(t)=Ax(t)+Bu(t) \\ &\qquad x(t_0)=x_0. \end{align*}$$

In [ ]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K, P

Choose weights.

In [ ]:
Q = np.diag([1., 1., 1.])
R = np.diag([1.])

Find optimal gain matrix (and cost matrix).

In [ ]:
K, P = lqr(A, B, Q, R)
print(f'K =\n{K}\n')

Find closed-loop eigenvalues.

In [ ]:
# Show closed-loop eigenvalues
print(f'p = {linalg.eigvals(A - B @ K)}')

Define a controller that maps sensor measurements to actuator commands. The `run` function will be called 100 times per second (i.e., at 100 Hz) by the simulator.

In [ ]:
class Controller:
    def __init__(self, K, q1e, v1e, v2e):
        self.K = K
        self.q1e = q1e
        self.v1e = v1e
        self.v2e = v2e
        self.taue = taue
    
    def reset(self):
        pass
    
    def run(
            self,
            t,
            platform_angle,
            platform_velocity,
            wheel_angle,
            wheel_velocity,
        ):
        
        x = np.array([
            platform_angle - self.q1e,
            platform_velocity - self.v1e,
            wheel_velocity - self.v2e,
        ])
        u = - self.K @ x

        wheel_torque = - (u[0] + self.taue)
        
        return wheel_torque

Create an instance of the controller.

In [ ]:
controller = Controller(K, q1e, v1e, v2e)

Reset the simulator. Optional arguments allow you to specify the initial wheel angle and velocity. If the value of either argument is `None` (or if the argument is left unspecified), then its value will be chosen at random.

In [ ]:
simulator.reset(
    platform_angle=np.pi + 0.2,
    platform_velocity=0.,
    wheel_angle=0.,
    wheel_velocity=5.,
)

Reset the controller.

In [ ]:
controller.reset()

Run the simulator.

In [ ]:
data = simulator.run(
    controller,           # <-- required (an instance of your Controller class)
    max_time=5.0,         # <-- optional (how long you want to run the simulation in seconds)
    data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
    video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
)

Plot the results.

In [ ]:
t = data['t']
q1 = data['platform_angle']
v1 = data['platform_velocity']
v2 = data['wheel_velocity']
tau = data['wheel_torque']
tau_command = data['wheel_torque_command']

fig, (ax_q1, ax_v1, ax_v2, ax_tau) = plt.subplots(4, 1, figsize=(8, 8), sharex=True)

ax_q1.plot(t, q1, label='$q_1$', linewidth=4)
ax_q1.plot(t, q1e * np.ones_like(t), '--', label='$q_{1e}$', linewidth=4)
ax_v1.plot(t, v1, label='$v_1$', linewidth=4)
ax_v1.plot(t, v1e * np.ones_like(t), '--', label='$v_{1e}$', linewidth=4)
ax_v2.plot(t, v2, label='$v_2$', linewidth=4)
ax_v2.plot(t, v2e * np.ones_like(t), '--', label='$v_{2e}$', linewidth=4)
ax_tau.plot(t, tau, label=r'$\tau$', linewidth=4)
ax_tau.plot(t, tau_command, '--', label=r'$\tau_{commanded}$', linewidth=4)
ax_tau.plot(t, np.ones_like(t) * simulator.tau_max, ':', label=r'$\tau_{max}$', linewidth=4, color='C2', zorder=0)
ax_tau.plot(t, - np.ones_like(t) * simulator.tau_max, ':', linewidth=4, color='C2', zorder=0)

ax_q1.grid()
ax_q1.legend(fontsize=16, ncol=2, loc='upper right')
ax_q1.tick_params(labelsize=14)
ax_q1.set_ylim(q1e - 0.5, q1e + 0.5)

ax_v1.grid()
ax_v1.legend(fontsize=16, ncol=2, loc='upper right')
ax_v1.tick_params(labelsize=14)
ax_v1.set_ylim(v1e - 1., v1e + 1.)

ax_v2.grid()
ax_v2.legend(fontsize=16, ncol=2, loc='upper right')
ax_v2.tick_params(labelsize=14)
ax_v2.set_ylim(v2e - 20., v2e + 20.)

ax_tau.grid()
ax_tau.legend(fontsize=16, ncol=3, loc='upper right')
ax_tau.tick_params(labelsize=14)

ax_tau.set_xlabel('time (s)', fontsize=20)
ax_tau.set_xlim([data['t'][0], data['t'][-1]])

fig.tight_layout()
plt.show()